We are working with the Public Use Files (PUFs) from the American Housing Survey (AHS) conducted by the US Census Bureau. These are very large files, so the primary order of business is simply selecting the columns we want from the large raw data import. To help us, we can use the column definitions from the AHS codebook (https://www.census.gov/data-tools/demo/codebook/ahs/ahsdict.html). We find the columns we want, and then extract them.

We start by performing the operations for the 2023 data, and once that's done, doing the same for prior years is simply a matter of running the script.

In [1]:
import pandas as pd
import numpy as np

In [2]:
test_dat = pd.read_csv('../../raw_data/AHS_2023_National_v1.1/household.csv')

There are a lot of columns to go through, and it's not necessarily worth processing all of them if we don't end up using them all. Below is a selection of columns that might be useful at first glance, which we isolate and process from the original set. Not all of the isolated columns are processed, but there is space to add processing if necessary. Additionally, some of the work could be reduced if we instead turn to the AHS codebook to find the keys when necessary, since there are little to no undocumented values in the dataset.

In [3]:
desired_cols = [
    # Location info.
    'OMB13CBSA',
    # Building information.
    'BLD', 'STORIES', 'YRBUILT',
    # Cost information.
    'RENT', 'RENTCNTRL', 'RENTSUB', 'UTILAMT', 'HOAAMT', 'INSURAMT', 'TOTHCAMT',
    # Unit information.
    'UNITSIZE', 'UNITFLOORS', 'BATHROOMS', 'BEDROOMS', 'TOTROOMS', 'KITCHENS', 'DINING', 'LAUNDY', 'GARAGE', 'PORCH',
    # Unit Amenities
    'ACPRIMARY', 'HEATTYPE', 'HOTWATER', 'SEWTYPE', 'WATSOURCE', 'COOKTYPE', 'FRIDGE', 'DISHWASH', 'WASHER', 'DRYER', 'KITCHSINK',
    # Maintenance information.
    'ADEQUACY', 'LEAKI', 'LEAKO', 'ROACH', 'RODENT', 'MOLDBASEM', 'MOLDBEDRM', 'MOLDBATH', 'MOLDKITCH', 'MOLDLROOM', 'MOLDOTHER',
    'NOTOIL', 'NOWAT', 'SEWBREAK', 'FLOORHOLE', 'ROOFHOLE', 'WALLCRACK',
    # Household information.
    'FINCP', 'HINCP', 'NUMNONREL', 'NUMPEOPLE', 'HHADLTKIDS', 'HHAGE', 'HHCITSHP', 'HHGRAD', 'HHMAR', 'HHMOVE', 
    'HHNATVTY', 'HHRACE', 'HHSEX', 'DISHH', 'HHPRNTHOME',
    # Neighborhood information.
    'SUBDIV', 'NEARABAND', 'NEARBARCL', 'NEARTRASH', 'RATINGNH', 'NHQPCRIME', 'NHQPUBTRN', 'NHQRISK', 'NHQSCHOOL', 'NHQSCRIME'
]

# Sex: 1 is Male, 2 is Female.
bin_cat_cols = ['RENTCNTRL', 'GARAGE', 'PORCH', 'FRIDGE', 'DISHWASH', 'WASHER', 'KITCHSINK', 'LEAKI', 
                'LEAKO', 'MOLDBASEM', 'MOLDBEDRM', 'MOLDBATH', 'MOLDKITCH', 'MOLDLROOM', 'MOLDOTHER',
                'NOTOIL', 'NOWAT', 'FLOORHOLE', 'ROOFHOLE', 'WALLCRACK', 'HHSEX', 'DISHH', 'SUBDIV',
                'NHQPCRIME', 'NHQPUBTRN', 'NHQRISK', 'NHQSCHOOL', 'NHQSCRIME']

def process_bin_cat(column: pd.Series):
    column = column.replace(-6, 'N/A')  # Replace -6 with N/A, as per documentation.
    column = column.replace(-9, None)  # Replace -9 with NaN.
    column = column.astype('category')
    column = column.cat.rename_categories({1.0: "Yes", 2.0: "No"})
    return column

def process_PUF(raw_dat:pd.DataFrame, year:int):
    raw_dat = raw_dat[desired_cols]
    raw_dat = raw_dat[raw_dat['RENT'] >= 0] # Remove rows corresponding to not rented houses.
    processed = raw_dat.copy()
    for col in bin_cat_cols:
        processed[col] = pd.Series([val.replace("'", '') for val in raw_dat[col]], dtype='float64')
        processed[col] = process_bin_cat(processed[col])
    processed['HHSEX'] = processed['HHSEX'].cat.rename_categories({'Yes':'Male', 'No':'Female'})
    # Bathrooms
    processed['BATHROOMS'] = raw_dat['BATHROOMS']
    processed['BATHROOMS'] = processed['BATHROOMS'].map({"'01'":'1', "'02'":'1.5', "'03'":'2', "'04'":'2.5', "'05'":'3', "'06'":'4', 
                                                        "'08'":'0.5', "'09'":'0.5', "'13'":'0'})
    processed['BATHROOMS'] = processed['BATHROOMS'].astype('float64')
    # Adequacy
    processed['ADEQUACY'] = raw_dat["ADEQUACY"].astype('category')
    processed['ADEQUACY'] = processed['ADEQUACY'].cat.rename_categories({"'1'":'Adequate', "'2'":'Moderately Inad', "'3'":'Severely Inad'})
    # Building type
    processed['BLD'] = [str(val).replace("'", '') for val in raw_dat['BLD']]
    processed['BLD'] = processed['BLD'].astype('category')
    processed['BLD'] = processed['BLD'].cat.rename_categories({
        '01': 'Trailer',
        '02': 'Single Family, Detached',
        '03': 'Single Family, Attached',
        '04': '2 Apts',
        '05': '3 to 4 Apts',
        '06': '5 to 9 Apts',
        '07': '10 to 19 Apts',
        '08': '20 to 49 Apts',
        '09': '50+ Apts',
        '10': 'Other',
    })
    # Rent subsidy
    processed['RENTSUB'] = [int(str(val).replace("'", '')) for val in raw_dat['RENTSUB']]
    processed['RENTSUB'] = processed['RENTSUB'].astype('category')
    processed['RENTSUB'] = processed['RENTSUB'].cat.rename_categories({
        1: "Public",
        2: "Portable voucher",
        3: "Non-portable voucher",
        4: "Other government subsidy",
        5: "Rent reduction requiring annual recertification not reported elsewhere",
        6: "Rent reduction because household member works for owner",
        7: "Rent reduction because household member related to owner",
        8: "None",
        -6: "N/A",
    })
    processed['RENTSUB'] = processed['RENTSUB'].cat.remove_categories(-9)
    # Remove negative values from numeric columns, excluding income.
    for col in processed.select_dtypes(['int64']).columns:
        if col not in ['FINCP', 'HINCP']:
            processed[col] = processed[col].replace(-9, np.nan)
    # Add the year.
    processed.insert(0, 'YEAR', year)
    return processed
    

In [4]:
AHS_2023_test = process_PUF(test_dat, 2023)

In [126]:
AHS_2023_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22835 entries, 0 to 55667
Data columns (total 75 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   YEAR        22835 non-null  int64   
 1   OMB13CBSA   22835 non-null  object  
 2   BLD         22835 non-null  category
 3   STORIES     22835 non-null  int64   
 4   YRBUILT     22835 non-null  int64   
 5   RENT        22835 non-null  int64   
 6   RENTCNTRL   8142 non-null   category
 7   RENTSUB     22211 non-null  category
 8   UTILAMT     22835 non-null  int64   
 9   HOAAMT      22815 non-null  float64 
 10  INSURAMT    22835 non-null  int64   
 11  TOTHCAMT    22835 non-null  int64   
 12  UNITSIZE    22835 non-null  object  
 13  UNITFLOORS  22835 non-null  int64   
 14  BATHROOMS   22835 non-null  float64 
 15  BEDROOMS    22835 non-null  int64   
 16  TOTROOMS    22835 non-null  int64   
 17  KITCHENS    22835 non-null  int64   
 18  DINING      22835 non-null  int64   
 19  LAUNDY   

In [ ]:
# Save as feather file to preserve data types.
AHS_2023_test.to_feather("../../data/AHS_2023.feather")